In [2]:
import torch
from torch import nn, optim
import os 
import torch.nn.functional as F

In [3]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
encoder.load_model(encoder_weights)
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

D:\pycharm_project\vs2ttsvoice\encoder\audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


Loaded encoder "pretrained.pt" trained to step 1564501
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder\saved_models\pretrained\pretrained.pt


In [4]:
def embedEX(in_fpath):
    filepath=r'D:\voiceset\dataset\test_wav'
#     filepath=r'D:\voiceset\dataset\wav'
    in_fpath=os.path.join(filepath,in_fpath[:7],in_fpath[8:-6],in_fpath[-5:]+'.wav')
    reprocessed_wav = encoder.preprocess_wav(in_fpath)
    original_wav, sampling_rate = librosa.load(in_fpath)
    preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
    embed,_ = encoder.embed_utterance(preprocessed_wav)
    return embed

In [5]:
class X2E_NetMSE3(torch.nn.Module):
    def __init__(self, in_dim, n_hidden_1,n_hidden_2,out_dim):
        super(X2E_NetMSE3, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Linear(in_dim, n_hidden_1), torch.nn.ReLU(True))
        self.layer2 = torch.nn.Sequential(torch.nn.Linear(n_hidden_1, n_hidden_2), torch.nn.ReLU(True))  
        self.layer3 = torch.nn.Sequential(torch.nn.Linear(n_hidden_2, out_dim))
 
    def forward(self, x):
        x=x/torch.norm(x, p=2)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x=x/torch.norm(x, p=2)
        return x

In [6]:
#模型
model=X2E_NetMSE3(512,400,300,256)
#损失函数
# criterion = torch.nn.MSELoss(reduce=True, size_average=False)
criterion = torch.nn.MSELoss(reduction='sum')
# criterion = torch.nn.L1Loss(reduction='sum')
#优化器
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
# 训练模型
epoch = 0
totaldir='D:\\pycharm_project\\vs2ttsvoice\\X2E'
#损失值写入文件
se_dir='D:\\pycharm_project\\vs2ttsvoice\\SE_vector'
for i in range(80):
    print(i)
    pathstr ='xvector'+str(i+1)+'.txt' 
    pathstr=os.path.join(totaldir,pathstr)
    sepath='dpdir'+str(i+1)+'.txt' 
    sepath=os.path.join(se_dir,sepath)
    se_vectors=open(sepath).readlines()
    file=open( pathstr)
    j=0
    for line in file.readlines():
        data=line.strip().replace('[','').replace(']','').split()
        se=se_vectors[j].replace('[','').replace(']','').split()
        se_filename=se[0]
        filename=data[0]
        if se_filename==filename:
#             print(se_filename)
            data=data[1:]
            se_data=se[1:]
            out=list(map(float,data))
            out=torch.tensor(out)
            pred=model(out)
            label=list(map(float,se_data))
            label=torch.tensor(label)
            loss = criterion(pred, label)
            print_loss = loss.data.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch+=1
            if epoch%100 == 0:
            #         print('label:{0}\n,out:{1}\n,pred:{2}\n'.format(label.size(),out.size(),pred.size()))
            #保存参数、
                print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
                state = { 'model':model.state_dict(), 'epoch':epoch }                       
                if not os.path.isdir('checkpointX2E'):
                     os.mkdir('checkpointX2E')    
                torch.save(state, '.\checkpointX2E\model3MSE_epoch_%d.ckpt' % (epoch))
                file_save = open('lossX2E3MSE.txt',mode='a')
                file_save.write('\n'+'epoch:'+str(epoch)+'  loss:'+str(loss.data.item()))
                file_save.close() 
        else:continue
        j=j+1

0
epoch: 100, loss: 0.1565
epoch: 200, loss: 0.2675
epoch: 300, loss: 0.08287
epoch: 400, loss: 0.4205
epoch: 500, loss: 0.3442
epoch: 600, loss: 0.1877
epoch: 700, loss: 0.1183
epoch: 800, loss: 0.2453
epoch: 900, loss: 0.2439
epoch: 1000, loss: 0.3134
epoch: 1100, loss: 0.4859
epoch: 1200, loss: 0.3899
epoch: 1300, loss: 0.4298
epoch: 1400, loss: 0.4807
epoch: 1500, loss: 0.337
epoch: 1600, loss: 0.1292
epoch: 1700, loss: 0.7113
epoch: 1800, loss: 0.3024
1
epoch: 1900, loss: 0.217
epoch: 2000, loss: 0.3056
epoch: 2100, loss: 0.149
epoch: 2200, loss: 0.6356
epoch: 2300, loss: 0.1234
epoch: 2400, loss: 0.3036
epoch: 2500, loss: 0.1442
epoch: 2600, loss: 0.2641
epoch: 2700, loss: 0.094
epoch: 2800, loss: 0.6148
epoch: 2900, loss: 0.1031
epoch: 3000, loss: 0.2621
epoch: 3100, loss: 0.1674
epoch: 3200, loss: 0.315
epoch: 3300, loss: 0.6109
epoch: 3400, loss: 0.3072
epoch: 3500, loss: 0.4147
epoch: 3600, loss: 0.2215
epoch: 3700, loss: 0.4092
2
epoch: 3800, loss: 0.6275
epoch: 3900, loss: 

epoch: 30800, loss: 0.202
17
epoch: 30900, loss: 0.1562
epoch: 31000, loss: 0.22
epoch: 31100, loss: 0.2911
epoch: 31200, loss: 0.3319
epoch: 31300, loss: 0.3428
epoch: 31400, loss: 0.2981
epoch: 31500, loss: 0.2573
epoch: 31600, loss: 0.2871
epoch: 31700, loss: 0.4078
epoch: 31800, loss: 0.2428
epoch: 31900, loss: 0.2982
epoch: 32000, loss: 0.1794
epoch: 32100, loss: 0.3624
epoch: 32200, loss: 0.3411
epoch: 32300, loss: 0.2581
epoch: 32400, loss: 0.2531
epoch: 32500, loss: 0.3404
epoch: 32600, loss: 0.2512
18
epoch: 32700, loss: 0.3005
epoch: 32800, loss: 0.1472
epoch: 32900, loss: 0.2411
epoch: 33000, loss: 0.3368
epoch: 33100, loss: 0.128
epoch: 33200, loss: 0.3703
epoch: 33300, loss: 0.3234
epoch: 33400, loss: 0.2769
epoch: 33500, loss: 0.2589
epoch: 33600, loss: 0.2161
epoch: 33700, loss: 0.4067
epoch: 33800, loss: 0.3077
epoch: 33900, loss: 0.2742
epoch: 34000, loss: 0.1386
epoch: 34100, loss: 0.1131
epoch: 34200, loss: 0.2055
epoch: 34300, loss: 0.2821
epoch: 34400, loss: 0.3876

epoch: 61100, loss: 0.2997
epoch: 61200, loss: 0.2366
epoch: 61300, loss: 0.2716
epoch: 61400, loss: 0.2424
epoch: 61500, loss: 0.1884
epoch: 61600, loss: 0.2061
epoch: 61700, loss: 0.2695
epoch: 61800, loss: 0.3186
epoch: 61900, loss: 0.1392
epoch: 62000, loss: 0.1561
epoch: 62100, loss: 0.38
epoch: 62200, loss: 0.1386
epoch: 62300, loss: 0.1621
34
epoch: 62400, loss: 0.2014
epoch: 62500, loss: 0.2197
epoch: 62600, loss: 0.1758
epoch: 62700, loss: 0.1696
epoch: 62800, loss: 0.1731
epoch: 62900, loss: 0.2014
epoch: 63000, loss: 0.2768
epoch: 63100, loss: 0.2202
epoch: 63200, loss: 0.2209
epoch: 63300, loss: 0.2078
epoch: 63400, loss: 0.2198
epoch: 63500, loss: 0.1704
epoch: 63600, loss: 0.1165
epoch: 63700, loss: 0.2451
epoch: 63800, loss: 0.2397
epoch: 63900, loss: 0.1065
epoch: 64000, loss: 0.2668
epoch: 64100, loss: 0.1178
35
epoch: 64200, loss: 0.2812
epoch: 64300, loss: 0.2924
epoch: 64400, loss: 0.1268
epoch: 64500, loss: 0.1362
epoch: 64600, loss: 0.2821
epoch: 64700, loss: 0.15

epoch: 91300, loss: 0.3109
epoch: 91400, loss: 0.1731
epoch: 91500, loss: 0.3095
epoch: 91600, loss: 0.2586
epoch: 91700, loss: 0.2265
epoch: 91800, loss: 0.1472
epoch: 91900, loss: 0.4341
epoch: 92000, loss: 0.1817
epoch: 92100, loss: 0.2724
51
epoch: 92200, loss: 0.2627
epoch: 92300, loss: 0.315
epoch: 92400, loss: 0.1733
epoch: 92500, loss: 0.3343
epoch: 92600, loss: 0.3126
epoch: 92700, loss: 0.2347
epoch: 92800, loss: 0.2016
epoch: 92900, loss: 0.1707
epoch: 93000, loss: 0.2419
epoch: 93100, loss: 0.2694
epoch: 93200, loss: 0.2351
epoch: 93300, loss: 0.2472
epoch: 93400, loss: 0.2217
epoch: 93500, loss: 0.2091
epoch: 93600, loss: 0.1483
epoch: 93700, loss: 0.1471
epoch: 93800, loss: 0.1728
epoch: 93900, loss: 0.2427
52
epoch: 94000, loss: 0.1253
epoch: 94100, loss: 0.2375
epoch: 94200, loss: 0.1579
epoch: 94300, loss: 0.198
epoch: 94400, loss: 0.174
epoch: 94500, loss: 0.2187
epoch: 94600, loss: 0.215
epoch: 94700, loss: 0.1675
epoch: 94800, loss: 0.2136
epoch: 94900, loss: 0.2288

epoch: 120800, loss: 0.1437
epoch: 120900, loss: 0.2321
epoch: 121000, loss: 0.3162
epoch: 121100, loss: 0.1432
epoch: 121200, loss: 0.2963
epoch: 121300, loss: 0.2259
epoch: 121400, loss: 0.2197
epoch: 121500, loss: 0.1321
epoch: 121600, loss: 0.2395
epoch: 121700, loss: 0.2086
66
epoch: 121800, loss: 0.3209
epoch: 121900, loss: 0.1288
epoch: 122000, loss: 0.2127
epoch: 122100, loss: 0.2645
epoch: 122200, loss: 0.2171
epoch: 122300, loss: 0.1217
epoch: 122400, loss: 0.1191
epoch: 122500, loss: 0.1889
epoch: 122600, loss: 0.1281
epoch: 122700, loss: 0.1749
epoch: 122800, loss: 0.2033
epoch: 122900, loss: 0.1807
epoch: 123000, loss: 0.1748
epoch: 123100, loss: 0.2985
epoch: 123200, loss: 0.09993
epoch: 123300, loss: 0.2161
epoch: 123400, loss: 0.2846
epoch: 123500, loss: 0.2194
67
epoch: 123600, loss: 0.1557
epoch: 123700, loss: 0.2391
epoch: 123800, loss: 0.2347
epoch: 123900, loss: 0.2142
epoch: 124000, loss: 0.1615
epoch: 124100, loss: 0.1446
epoch: 124200, loss: 0.2178
epoch: 124300

In [8]:
# 读取训练参数
# checkpoint = torch.load(r'C:\NLP\tts_rtvc\rtvc\checkpoint\model3_epoch_56300.ckpt')
checkpoint = torch.load(r'D:\pycharm_project\vs2ttsvoice\checkpointX2E\model3MSE_epoch_148600.ckpt')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [9]:
#合成语音
from IPython.utils import io
epoch=0
text="This is being said in my own voice.The computer has learned to do an impression of me."
file=open('.\X2E\pv_xvector.txt')
filenamelabel=''
for line in file.readlines():
# line=file.readline()
    data=line.strip().replace('[','').replace(']','').split()
    filename=data[0]
    if (filenamelabel==filename[:7]):
        continue
    print(filename)
    data=data[1:]
#     out=[[list(map(float,data))]]
#     out=torch.tensor(out)
#     out=out.permute(0,2,1)
#     pred=model(out)
#     pred=pred.squeeze(0)
#     pred=pred.tolist()
#     print("pred",pred)
    out=list(map(float,data))
    out=torch.tensor(out)
    pred=model(out)
    pred=pred.tolist()
#     print("pred",pred)
    print("Synthesizing 预测语音...")
    with io.capture_output() as captured:
      specs = synthesizer.synthesize_spectrograms([text], [pred])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
#     sf.write(filename[:7]+"A.wav", generated_wav, synthesizer.sample_rate)
    epoch =+1
    #原始声音合成
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
    label=embedEX(filename)
    print("Synthesizing 原始语音...")
    print('阈值：',np.linalg.norm(pred-label))
    with io.capture_output() as captured:
       specs = synthesizer.synthesize_spectrograms([text], [label])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    filenamelabel=filename[:7]
#     sf.write(filename[:7]+".wav", generated_wav, synthesizer.sample_rate)

id10270-5r0dWxy17C8-00001
Synthesizing 预测语音...
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the con

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.513019507232136


id10271-1gtz-CUIygI-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.5407127300117605


id10272-0qqQJVFKpbA-00001
Synthesizing 预测语音...


...

id10273-0LbtndiXJC0-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.49711042861710764


id10274-7rkVfW2yw0c-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.5005684498437454


KeyboardInterrupt: 